In [19]:
import duckdb

path = "/Users/mz/Downloads/133957095_NewYork-PresbyterianHospital_standardcharges 2.json"

con = duckdb.connect()

# query = f"""
# WITH root AS (
#     SELECT standard_charge_information
#     FROM read_json_auto('{path}',maximum_object_size=1000000000)
# ),

# sci AS (
#     SELECT UNNEST(standard_charge_information) AS sci
#     FROM root
# ),

# -- flatten CPT/HCPCS codes
# codes AS (
#     SELECT
#         sci.description,
#         UNNEST(sci.code_information) AS ci
#     FROM sci
# ),

# -- flatten payer-level prices
# charges AS (
#     SELECT
#         sci.description,
#         UNNEST(sci.standard_charges) AS sc
#     FROM sci
# ),

# payers AS (
#     SELECT
#         description,
#         sc.setting,
#         sc.gross_charge,
#         sc.discounted_cash,
#         UNNEST(sc.payers_information) AS pi
#     FROM charges
# )

# -- Join CPT codes to payer prices by shared description
# SELECT
#     ci.code               AS cpt_code,
#     ci.type               AS code_type,
#     p.setting,
#     p.gross_charge,
#     p.discounted_cash,
#     p.pi.payer_name,
#     p.pi.pIdx,
#     p.pi.plan_name,
#     p.pi.standard_charge_percentage,
#     p.pi.billing_class,
#     p.pi.methodology,
#     p.pi.estimated_amount
# FROM codes ci
# JOIN payers p
# ON ci.description = p.description
# where ci.code = '92250' and ci.type = 'CPT'
# """


In [28]:


query = """  SELECT
        sci.description,
        sci.drug_information.unit      AS drug_unit,
        sci.drug_information.type      AS drug_type,
        code.code                      AS code,
        code.type                      AS code_type,
        sc.setting                     AS setting,
        sc.gross                       AS gross_charge,
        sc.discounted_cash,
        sc.minimum,
        sc.maximum,
        payer.payer_name,
        payer.plan_name,
        payer.standard_charge_dollar,
        payer.methodology,
        payer.additional_payer_notes
    FROM read_json_auto('{path}',maximum_object_size=1000000000),
         UNNEST(standard_charge_information) AS sci
         LEFT JOIN UNNEST(sci.code_information)        AS code
         LEFT JOIN UNNEST(sci.standard_charges)        AS sc
         LEFT JOIN UNNEST(sc.payers_information)       AS payer
"""

In [29]:

df = con.execute(query).fetchdf()
print(df.head())


ParserException: Parser Error: syntax error at end of input

In [ ]:
df

,cpt_code,code_type,setting,gross_charge,discounted_cash,payer_name,pIdx,plan_name,standard_charge_percentage,billing_class,methodology,estimated_amount
0,92250,CPT,outpatient,NaN,NaN,Longevity Health Plan,3446,MA,NaN,facility,fee schedule,NaN
1,92250,CPT,outpatient,NaN,NaN,United Healthcare,1845,Compass,21.0,facility,percent of total billed charges,999999999.0
2,92250,CPT,outpatient,NaN,NaN,Brighton,4107,CreateIDN,70.3,facility,percent of total billed charges,999999999.0
3,92250,CPT,outpatient,NaN,NaN,Health Plus,3494,MEDICAID,NaN,facility,fee schedule,NaN
4,92250,CPT,outpatient,NaN,NaN,Affinity Health Plan,164,MEDICAID,NaN,facility,fee schedule,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
64,92250,CPT,outpatient,NaN,NaN,Health Plus,3494,CHILDHEALTHPLUS,NaN,facility,fee schedule,NaN
65,92250,CPT,outpatient,NaN,NaN,Coventry,3122,FirstHealthPPO,85.0,facility,percent of total billed charges,999999999.0
66,92250,CPT,outpatient,NaN,NaN,Fidelis Care New York,52,HARP,NaN,facility,fee schedule,NaN
67,92250,CPT,outpatient,NaN,NaN,Health First,3429,ESSENTIALPL1 and 2,NaN,facility,fee schedule,NaN


In [ ]:
cpt_values = (
    df.dropna(subset=["estimated_amount"])
      .query("estimated_amount < 1e7")
      .groupby("cpt_code")["estimated_amount"]
      .median()
      .reset_index(name="cpt_value")
)


In [ ]:
cpt_values

,cpt_code,cpt_value
